In [1]:
!./setup.sh

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 98 kB 8.1 MB/s eta 0:00:011
     |████████████████████████████████| 38.2 MB 925 kB/s eta 0:00:011     |█████████████████████▏          | 25.3 MB 498 kB/s eta 0:00:26
     |████████████████████████████████| 12.4 MB 993 kB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 44.8 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 24.7 MB/s eta 0:00:01
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 2.2 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 27.2 MB/s eta 0:00:01
     |████████████████████████████████| 722 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 46.3

In [3]:
# 采用asr的结果
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|██████████████████████████████| 141/141 [01:36<00:00,  1.47it/s]
epoch(0)(video):  0.6482096685392892
epoch(0)(aud

In [5]:
# 采用CLR
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|██████████████████████████████| 141/141 [01:25<00:00,  1.66it/s]
epoch(0)(video):  0.6453660831056209
epoch(0)(aud

In [ ]:
# early fusion 去掉audio
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|██████████████████████████████| 141/141 [01:24<00:00,  1.68it/s]
epoch(0)(video):  0.6878608662219391
epoch(0)(aud

In [2]:
!python -W ignore ensemble_inference.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.

In [ ]:
!python -W ignore end2end_main.py

In [1]:
# without projector
!python -W ignore enhance_main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|██████████████████████████████| 157/157 [04:32<00:00,  1.74s/it]
6.818717795572463
train (1): 100%|███████████████

In [1]:
# 使用eff特征，自监督 rd = 2023
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|██████████████████████████████| 141/141 [02:12<00:00,  1.07it/s]
epoch(0)(video):  0.689400477257959
epoch(0)(audi

In [ ]:
!python -W ignore inference.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming


In [ ]:
# ensemble 模型在验证集上测试
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'
import yaml
import json
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import utils.train_util as train_util
from dataloader.dataloader import TestingDataset
from src.loss.loss_compute import SimpleLossCompute
from src.model.baseline_model import Baseline
from src.loop.run_epoch import training_loop,validating_loop
from dataloader.dataloader import MultimodaFeaturesDataset,Datasetfortextcnn
from torch.utils.data import DataLoader
batch_size = 8
modal_name_list = ['video','audio','text']
config_path = './config/config.yaml'
config = yaml.load(open(config_path))
dataset = MultimodaFeaturesDataset(config['DatasetConfig'],job='valdation')
loader = DataLoader(dataset,num_workers=8,
                    batch_size=batch_size,
                    pin_memory=False,
                    collate_fn=dataset.collate_fn)

model_path_1 = '../checkpoint/0609/01/epoch_48 0.7888.pt'
model_path_2 = '../checkpoint/0609/01/epoch_30 0.7886.pt'
model_path_3 = '../checkpoint/0609/01/epoch_24 0.7871.pt'

model_path_4 = '../checkpoint/0609/02/epoch_46 0.7868.pt'
model_path_5 = '../checkpoint/0609/02/epoch_28 0.7856.pt'

model_path_6 = '../checkpoint/0608/01/epoch_86 0.7877.pt'
model_path_7 = '../checkpoint/0608/01/epoch_50 0.7873.pt'
model_path_8 = '../checkpoint/0608/01/epoch_28 0.7869.pt'

model_path_9 = '../checkpoint/0608/03/epoch_28 0.7877.pt'
model_path_10 = '../checkpoint/0608/03/epoch_52 0.7890.pt'
model_path_11 = '../checkpoint/0608/03/epoch_74 0.7896.pt'
models_path = [model_path_1,
               model_path_4,
               model_path_6,
               model_path_10,model_path_11]
model_weights = [0.2,0.1,0.2,0.25,0.25] #0.791
# model_weights = [0.1,0.1,0.1,0.35,0.35] # 0.789
# model_weights = [0.2,0.2,0.2,0.2,0.2] # 0.7911
#model_weights = np.array(np.random.random(11))
#model_weights = model_weights/sum(model_weights)
device = 'cuda'
top_k=20
# output_json = './0604_resnet_ensemble.json'
models = []
for path in models_path:
    if(path.split('/')[2]+path.split('/')[3]=='060801'):
        config['ModelConfig']['fusion_head_params']['concat_feat_dim']['fusion'] = 30720
        config['ModelConfig']['audio_head_params']['max_frames'] = 300
    else:
        config['ModelConfig']['fusion_head_params']['concat_feat_dim']['fusion'] = 29696
        config['ModelConfig']['audio_head_params']['max_frames'] = 200
    model = Baseline(config['ModelConfig'])
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval()
    models.append(model)

tagging_class_num = 82
evl_metrics = [train_util.EvaluationMetrics(tagging_class_num, top_k=20)
                           for i in range(len(modal_name_list)+1)] #+1 for fusion
for i in range(len(evl_metrics)):
    evl_metrics[i].clear()
metric_dict = {}
gap_dict = {}
with torch.no_grad():
    for i,batch in tqdm(enumerate(loader)):
        if(len(batch)==5):
            video,audio,text,text_mask,label = batch
            video = video.to(device)
            audio = audio.to(device)
            text = text.to(device)
            text_mask = text_mask.to(device)
            label = label.to(device)
        else:
            video,audio,text,label = batch
            video = video.to(device)
            audio = audio.to(device)
            text = text.to(device)
            label = label.to(device)

        inputs_dict={}
        inputs_dict['video'] = video
        inputs_dict['audio'] = audio
        inputs_dict['text'] = text 
        if(len(batch)==5):
            inputs_dict['attention_mask'] = text_mask
        else:
            inputs_dict['attention_mask'] = None

        B = video.shape[0]
        pred_dict_ensemble = {}
        for modal_name in (modal_name_list+['fusion']):
            pred_dict_ensemble['tagging_output_'+modal_name] = {}
            pred_dict_ensemble['tagging_output_'+modal_name]['predictions'] = torch.zeros(B,82).cuda()

        for i,model in enumerate(models):
            pred_dict = model(inputs_dict)
            for modal_name in (modal_name_list+['fusion']):
                pred_dict_ensemble['tagging_output_'+modal_name]['predictions'] += model_weights[i]*pred_dict['tagging_output_'+modal_name]['predictions']
        '''
        for modal_name in (modal_name_list+['fusion']):
            pred_dict_ensemble['tagging_output_'+modal_name]['predictions'] = pred_dict_ensemble['tagging_output_'+modal_name]['predictions']/len(models)
        '''
        for index,modal_name in enumerate(modal_name_list+['fusion']):
            pred = pred_dict_ensemble['tagging_output_'+modal_name]
            pred = pred['predictions'].detach().cpu().numpy()
            val_label = label.cpu().numpy()
            gap = train_util.calculate_gap(pred, val_label)
            evl_metrics[index].accumulate(pred, val_label, loss=0)
    for index,modal_name in enumerate(modal_name_list+['fusion']):
        metric_dict[modal_name] = evl_metrics[index].get()
        gap_dict[modal_name] = metric_dict[modal_name]['gap']
    print(gap_dict)



/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/ipykernel/__main__.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initia

initialization: Kaiming


Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


initialization: Kaiming


Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


initialization: Kaiming


Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


initialization: Kaiming


/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to N

In [ ]:
!python -W ignore main.py

In [4]:
import linecache
data_num_per_sample = 6
train_file = '../dataset/tagging/GroundTruth/datafile/train_resnet.txt'
val_file = '../dataset/tagging/GroundTruth/datafile/val_resnet.txt'
train_full_file = '../dataset/tagging/GroundTruth/datafile/train_full_resnet.txt'

In [6]:
with open(train_full_file,'w') as f:
    for index in range(4500):
        data_list = []
        for line_i in range(data_num_per_sample*index+1,data_num_per_sample*(index+1)):
            line = linecache.getline(train_file,line_i)
            # line = line.strip('\r\n')
            data_list.append(line)
        for line in data_list:
            f.write(line)
        f.write('\r\n')

In [7]:
with open(train_full_file,'a') as f:
    for index in range(500):
        data_list = []
        for line_i in range(data_num_per_sample*index+1,data_num_per_sample*(index+1)):
            line = linecache.getline(val_file,line_i)
            # line = line.strip('\r\n')
            data_list.append(line)
        for line in data_list:
            f.write(line)
        f.write('\r\n')

In [2]:
import torch
m = torch.load('../checkpoint/0607/enhance_VT/30_wp.pt',map_location=torch.device('cpu'))

In [5]:
state_dict = {}
for k,v in m.items():
    state_dict[k] = v.shape
    print(k,v.shape)

fusion_head_dict.video.hidden1_weights torch.Size([28672, 1024])
fusion_head_dict.video.gating_weights_1 torch.Size([1024, 128])
fusion_head_dict.video.gating_weights_2 torch.Size([128, 1024])
fusion_head_dict.video.bn_1.weight torch.Size([1024])
fusion_head_dict.video.bn_1.bias torch.Size([1024])
fusion_head_dict.video.bn_1.running_mean torch.Size([1024])
fusion_head_dict.video.bn_1.running_var torch.Size([1024])
fusion_head_dict.video.bn_1.num_batches_tracked torch.Size([])
fusion_head_dict.video.bn_2.weight torch.Size([128])
fusion_head_dict.video.bn_2.bias torch.Size([128])
fusion_head_dict.video.bn_2.running_mean torch.Size([128])
fusion_head_dict.video.bn_2.running_var torch.Size([128])
fusion_head_dict.video.bn_2.num_batches_tracked torch.Size([])
fusion_head_dict.audio.hidden1_weights torch.Size([1024, 1024])
fusion_head_dict.audio.gating_weights_1 torch.Size([1024, 128])
fusion_head_dict.audio.gating_weights_2 torch.Size([128, 1024])
fusion_head_dict.audio.bn_1.weight torch.Si

In [6]:
import torch
c = torch.load('../checkpoint/0608/enhance_VT_lr/50_wp.pt',map_location=torch.device('cpu'))

In [7]:
c_state_dict = {}
for k,v in c.items():
    c_state_dict[k] = v.shape
    print(k,v.shape)

fusion_head_dict.video.hidden1_weights torch.Size([28672, 1024])
fusion_head_dict.video.gating_weights_1 torch.Size([1024, 128])
fusion_head_dict.video.gating_weights_2 torch.Size([128, 1024])
fusion_head_dict.video.bn_1.weight torch.Size([1024])
fusion_head_dict.video.bn_1.bias torch.Size([1024])
fusion_head_dict.video.bn_1.running_mean torch.Size([1024])
fusion_head_dict.video.bn_1.running_var torch.Size([1024])
fusion_head_dict.video.bn_1.num_batches_tracked torch.Size([])
fusion_head_dict.video.bn_2.weight torch.Size([128])
fusion_head_dict.video.bn_2.bias torch.Size([128])
fusion_head_dict.video.bn_2.running_mean torch.Size([128])
fusion_head_dict.video.bn_2.running_var torch.Size([128])
fusion_head_dict.video.bn_2.num_batches_tracked torch.Size([])
fusion_head_dict.audio.hidden1_weights torch.Size([1024, 1024])
fusion_head_dict.audio.gating_weights_1 torch.Size([1024, 128])
fusion_head_dict.audio.gating_weights_2 torch.Size([128, 1024])
fusion_head_dict.audio.bn_1.weight torch.Si

In [6]:
# 0615/01 多卡试试 更大的batch size
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|████████████████████████████████| 47/47 [00:58<00:00,  1.25s/it]
epoch(0)(video):  0.6585780047869804
epoch(0)(aud

In [11]:
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|████████████████████████████████| 44/44 [00:46<00:00,  1.06s/it]
epoch(0)(video):  0.6616481288095377
epoch(0)(aud

In [ ]:
# 修复mask的自监督
!python -W ignore enhance_main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|████████████████████████████████| 79/79 [01:29<00:00,  1.13s/it]
12.190704593175575
train (1): 100%|██████████████

In [4]:
# 修复mask的自监督 + 修复mask的main
!python -W ignore main.py

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|████████████████████████████████| 47/47 [01:24<00:00,  1.81s/it]
epoch(0)(video):  0.6570029053911018
epoch(0)(aud

In [ ]:
import os

dataset_root = '../dataset/videos/video_5k/train_5k/'

# ########## get train_5k_A video file lists
videos_train_5k_A_dir = os.path.join(dataset_root, 'videos/train_5k_A')
videos_train_5k_A_files = [os.path.join(videos_train_5k_A_dir, f) for f in os.listdir(videos_train_5k_A_dir) if os.path.isfile(os.path.join(videos_train_5k_A_dir, f))]

print("videos_train_5k_A_dir= {}".format(videos_train_5k_A_dir))
print("len(videos/train_5k_A)= {}".format(len(videos_train_5k_A_files)))

# ########## display
from IPython.display import display, HTML

# video
test_video_path = videos_train_5k_A_files[3000]
print(test_video_path)
print(os.path.exists(test_video_path))
html_str = '''
<video controls width=\"500\" height=\"500\" src=\"{}\">animation</video>
'''.format(test_video_path)
print(html_str)
display(HTML(html_str))

In [3]:
!python -W ignore main.py   --device_ids 0 \
                            --pretrained_model ../checkpoint/0616/enhance_VT/50_wop.pt \
                            --saved_path ../checkpoint/0616/01/

Some weights of the model checkpoint at ../pretrained/bert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
initialization: Kaiming
train (0): 100%|████████████████████████████████| 94/94 [01:55<00:00,  1.23s/it]
epoch(0)(video):  0.6840189556313095
epoch(0)(aud